In [1]:
import os
from datetime import datetime

import ipyparallel
import pandas as pd
import databricks.koalas as ks

In [2]:
# download the dataset onto the machine (18GB)
!gsutil -m cp -r gs://bitcoin_bucket_20200308/data .

Copying gs://bitcoin_bucket_20200308/data/ADA-BTC.csv...
Copying gs://bitcoin_bucket_20200308/data/ADX-BTC.csv...                        
Copying gs://bitcoin_bucket_20200308/data/AE-BTC.csv...                         
Copying gs://bitcoin_bucket_20200308/data/AION-BTC.csv...                       
Copying gs://bitcoin_bucket_20200308/data/AGI-BTC.csv...                        
Copying gs://bitcoin_bucket_20200308/data/ALGO-BTC.csv...                       
Copying gs://bitcoin_bucket_20200308/data/ANKR-BTC.csv...                       
Copying gs://bitcoin_bucket_20200308/data/AMB-BTC.csv...
Copying gs://bitcoin_bucket_20200308/data/APPC-BTC.csv...                       
Copying gs://bitcoin_bucket_20200308/data/ARDR-BTC.csv...                       
Copying gs://bitcoin_bucket_20200308/data/ARK-BTC.csv...                        
Copying gs://bitcoin_bucket_20200308/data/ARPA-BTC.csv...                       
Copying gs://bitcoin_bucket_20200308/data/ARN-BTC.csv...                    

In [3]:
def build_frame_pandas(base_path):
    df = pd.DataFrame([])
    filenames = os.listdir(base_path)
    n_count = len(filenames)
    for n, filename in enumerate(filenames, 1):
        full_path = f'{base_path}/{filename}'
        name = filename.split('-')[0]
        new = pd.read_csv(full_path)
        new['open_time'] = pd.to_datetime(
            new['open_time'],
            unit='ms'
        )
        new = new.set_index('open_time', drop=True)
        new[name] = (new['open'] + new['close']) / 2
        df = df.join(new[name], how='outer')
        print(f'Joined {filename} ({n}/{n_count})')
    return df

In [4]:
def build_frame_koalas(base_path):
    first_run = True
    filenames = os.listdir(base_path)
    n_count = len(filenames)
    for n, filename in enumerate(filenames, 1):
        full_path = f'{base_path}/{filename}'
        name = filename.split('-')[0]
        new = ks.read_csv(full_path)
        new['open_time'] = ks.to_datetime(
            new['open_time'],
            unit='ms'
        )
        new = new.set_index('open_time', drop=True)
        new[name] = (new['open'] + new['close']) / 2
        if first_run:
            df = ks.DataFrame(new[name])
            first_run = False
        else:
            df = df.join(new[name], how='outer')
        print(f'Joined {filename} ({n}/{n_count})')
    return df

In [ ]:
# Binder's nodes have a memory capacity of at least 1GB and at most 2GB
# swap memory is not enabled by default
# !!!so this block of code actually kills the kernel in single node mode!!!

start = datetime.now()
df = build_frame_pandas('data')

print(datetime.now() - start)
print(df.info())
print(df.resample('1min').asfreq().ffill().info())

In [5]:
# to enable parallel computing:
# (1) go back to the JupyerHub's home page
# (2) create a new terminal and run `ipcluster start -n 4`
# (3) click the "IPython Clusters" tab on the JupyterHub's home page
# (4) attach a parallel profile to the kernel by pressing refresh and start

In [6]:
# let's verify the amount of engines running and available to our kernel
ipyparallel.Client().ids

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [8]:
start = datetime.now()
df = build_frame_koalas('data')

print(datetime.now() - start)
print(df.info())
print(df.resample('1min').asfreq().ffill().info())

Joined GO-BTC.csv (1/198)
Joined REN-BTC.csv (2/198)
Joined RDN-BTC.csv (3/198)
Joined PIVX-BTC.csv (4/198)
Joined CDT-BTC.csv (5/198)
Joined BTT-BTC.csv (6/198)
Joined SALT-BTC.csv (7/198)
Joined DGD-BTC.csv (8/198)
Joined ERD-BTC.csv (9/198)
Joined NANO-BTC.csv (10/198)
Joined WIN-BTC.csv (11/198)
Joined WAVES-BTC.csv (12/198)
Joined EDO-BTC.csv (13/198)
Joined DCR-BTC.csv (14/198)
Joined ZRX-BTC.csv (15/198)
Joined AMB-BTC.csv (16/198)
Joined TROY-BTC.csv (17/198)
Joined SNGLS-BTC.csv (18/198)
Joined MCO-BTC.csv (19/198)
Joined KEY-BTC.csv (20/198)
Joined ANKR-BTC.csv (21/198)
Joined BQX-BTC.csv (22/198)
Joined PERL-BTC.csv (23/198)
Joined MATIC-BTC.csv (24/198)
Joined HBAR-BTC.csv (25/198)
Joined INS-BTC.csv (26/198)
Joined NAV-BTC.csv (27/198)
Joined ADX-BTC.csv (28/198)
Joined MOD-BTC.csv (29/198)
Joined ARDR-BTC.csv (30/198)
Joined BCD-BTC.csv (31/198)
Joined LEND-BTC.csv (32/198)
Joined YOYO-BTC.csv (33/198)
Joined ICX-BTC.csv (34/198)
Joined AST-BTC.csv (35/198)
Joined AE-BTC.

Py4JJavaError: An error occurred while calling o241569.collectToPython.
: java.lang.StackOverflowError
	at org.apache.spark.sql.execution.SparkPlan.prepare(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$prepare$1.apply(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$prepare$1.apply(SparkPlan.scala:200)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.execution.SparkPlan.prepare(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$prepare$1.apply(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$prepare$1.apply(SparkPlan.scala:200)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.execution.SparkPlan.prepare(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$prepare$1.apply(SparkPlan.scala:200)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$prepare$1.apply(SparkPlan.scala:200)
	at scala